<a href="https://colab.research.google.com/github/DeepLearning-Y4/TrafficSignRecognition/blob/main/TrafficSignRecognition_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TrafficSignRecognition_CNN_model

### Load and Explore Dataset

The dataset already has preprocessed training and testing .p (pickle) files.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

### Unzip the dataset

In [ ]:
# Unzip the dataset
import zipfile
import os

zip_path = '/content/drive/MyDrive/archive.zip'
extract_path = '/content/traffic_signs'

if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/content/')
    print("Dataset extracted successfully!")
else:
    print("Dataset already extracted!")

### Load and Explore Dataset

In [ ]:
# ===== Load and Explore Dataset =====
import pickle
import numpy as np
import matplotlib.pyplot as plt

# Load training and testing data
with open('/content/traffic_signs/train.p', 'rb') as f:
    train_data = pickle.load(f)
with open('/content/traffic_signs/valid.p', 'rb') as f:
    valid_data = pickle.load(f)
with open('/content/traffic_signs/test.p', 'rb') as f:
    test_data = pickle.load(f)

# Extract images and labels
X_train, y_train = train_data['features'], train_data['labels']
X_valid, y_valid = valid_data['features'], valid_data['labels']
X_test, y_test = test_data['features'], test_data['labels']

print('Training set:', X_train.shape)
print('Validation set:', X_valid.shape)
print('Test set:', X_test.shape)
print('Number of classes:', len(np.unique(y_train)))

Preview some images:

In [ ]:
# Preview some images
fig, axs = plt.subplots(2, 5, figsize=(12, 5))
for i in range(10):
    axs[i//5, i%5].imshow(X_train[i])
    axs[i//5, i%5].set_title(f"Label: {y_train[i]}")
    axs[i//5, i%5].axis('off')
plt.tight_layout()
plt.show()

### Data Preprocessing
Normalize and convert to grayscale (optional but improves performance):

In [ ]:
# ===== Data Preprocessing =====
import cv2

# Convert to grayscale
print("Converting to grayscale...")
X_train_gray = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in X_train])
X_valid_gray = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in X_valid])
X_test_gray  = np.array([cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in X_test])

# Normalize pixel values to [0, 1]
X_train_gray = X_train_gray / 255.0
X_valid_gray = X_valid_gray / 255.0
X_test_gray  = X_test_gray / 255.0

# Reshape for CNN input (N, H, W, 1)
X_train_gray = X_train_gray.reshape(X_train_gray.shape + (1,))
X_valid_gray = X_valid_gray.reshape(X_valid_gray.shape + (1,))
X_test_gray  = X_test_gray.reshape(X_test_gray.shape + (1,))

print("Preprocessing complete!")
print("Training shape:", X_train_gray.shape)


### Build the CNN Model
We'll build a simple CNN similar to LeNet architecture:

In [ ]:
# ===== Build Optimized CNN Model =====
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

model = models.Sequential([
    # First Convolutional Block
    layers.Conv2D(32, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.001),
                  input_shape=(32,32,1)),
    layers.BatchNormalization(),

    layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    # Second Convolutional Block
    layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    # Third Convolutional Block
    layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),

    # Fully Connected Layers
    layers.Flatten(),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(43, activation='softmax')  # 43 traffic sign classes
])

model.summary()


### Compile and Train the Model



In [ ]:
# ===== Compile Model =====
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# ===== Set up Callbacks =====
# Early stopping to prevent overfitting
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=8,
    restore_best_weights=True,
    verbose=1
)

# Reduce learning rate when validation loss plateaus
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=4,
    min_lr=1e-7,
    verbose=1
)

# ===== Train Model =====
print("\nTraining the model...")
history = model.fit(
    X_train_gray, y_train,
    epochs=50,
    validation_data=(X_valid_gray, y_valid),
    batch_size=128,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


### Evaluate Performance

In [ ]:
# ===== Evaluate Performance =====
print("\nEvaluating on test set...")
test_loss, test_acc = model.evaluate(X_test_gray, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")

# Evaluate on training and validation sets for comparison
train_loss, train_acc = model.evaluate(X_train_gray, y_train, verbose=0)
val_loss, val_acc = model.evaluate(X_valid_gray, y_valid, verbose=0)

print(f"\nTraining Accuracy: {train_acc:.4f} ({train_acc*100:.2f}%)")
print(f"Validation Accuracy: {val_acc:.4f} ({val_acc*100:.2f}%)")
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")

Plot accuracy/loss curves:

In [ ]:
# ===== Plot Training History =====
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot
ax1.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.set_title('Training vs Validation Accuracy', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Loss plot
ax2.plot(history.history['loss'], label='Train Loss', linewidth=2)
ax2.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Loss', fontsize=12)
ax2.set_title('Training vs Validation Loss', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Confusion Matrix & Classification Report

In [ ]:
# Classification Report
print("\n" + "="*60)
print("CLASSIFICATION REPORT")
print("="*60)
print(classification_report(y_test, y_pred, digits=4))

# ===== Performance Metrics Summary =====
print("\n" + "="*60)
print("PERFORMANCE METRICS SUMMARY")
print("="*60)

metrics_dict = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'Precision (Macro)': precision_score(y_test, y_pred, average='macro', zero_division=0),
    'Recall (Macro)': recall_score(y_test, y_pred, average='macro', zero_division=0),
    'F1-Score (Macro)': f1_score(y_test, y_pred, average='macro', zero_division=0),
    'Precision (Weighted)': precision_score(y_test, y_pred, average='weighted', zero_division=0),
    'Recall (Weighted)': recall_score(y_test, y_pred, average='weighted', zero_division=0),
    'F1-Score (Weighted)': f1_score(y_test, y_pred, average='weighted', zero_division=0)
}

for metric, value in metrics_dict.items():
    print(f"{metric:.<40} {value:.4f} ({value*100:.2f}%)")


### Save the Model

In [ ]:
model.save('/content/drive/MyDrive/traffic_sign_cnn_optimized.h5')
print("\nModel saved successfully to Google Drive!")

### Save results to a text file

In [ ]:
# Save results to a text file
with open('/content/drive/MyDrive/cnn_results.txt', 'w') as f:
    f.write("CNN Model Performance Results\n")
    f.write("="*60 + "\n\n")
    f.write(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)\n")
    f.write(f"Test Loss: {test_loss:.4f}\n\n")
    for metric, value in metrics_dict.items():
        f.write(f"{metric}: {value:.4f} ({value*100:.2f}%)\n")

print("Results saved to cnn_results.txt in Google Drive!")